In [ ]:
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms

In [ ]:
# Introduce parameters
batch_size = 64
num_classes = 10
lr = 0.001
num_epochs = 10

# Loading Dataset
train_dataset = torchvision.datasets.MNIST(
    root="./data",
    train=True,
    transform = transforms.Compose([
        transforms.Resize((32,32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.1307), std = (0.3081))
    ])
, download=True)

test_dataset = torchvision.datasets.MNIST(
    root="./data",
    train= False,
    transform = transforms.Compose([
        transforms.Resize((32,32)),
        transforms.ToTensor(),
        transforms.Normalize(mean =(0.1325), std=(0.1305))
    ])
, download=True)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 58.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 2.07MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.6MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.34MB/s]


In [ ]:
# check dataset size
print('train dataset size', len(train_dataset))
print('test dataset size', len(test_dataset))

train dataset size 60000
test dataset size 10000


In [ ]:
# Defining Convolutional Neural Network
# Functional API

class LeNet5(nn.Module):
  def __init__(self, num_classes):
    super(LeNet5, self).__init__()

    self.layer1 = nn.Sequential(
        nn.Conv2d(1,6, kernel_size=5, stride=1, padding=0),
        nn.BatchNorm2d(6),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride =2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(6,16, kernel_size = 5, stride=1, padding=0),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.fc = nn.Linear(400, 120)
    self.relu = nn.ReLU()
    self.fc1 = nn.Linear(120, 84)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(84, num_classes)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.reshape(out.size(0), -1)
    out = self.fc(out)
    out = self.relu(out)
    out = self.fc1(out)
    out = self.relu1(out)
    out = self.fc2(out)
    return out

model = LeNet5(num_classes)

# Defining Loss Calculation function
cost = nn.CrossEntropyLoss()

# Defining Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [ ]:
# Model Training
total_step = len(train_loader)

for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images
    labels = labels
    # Forward pass
    outputs = model(images)
    loss = cost(outputs, labels)

    # Back prop and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch: {epoch+1}/{num_epochs}, Step: {i+1}/{total_step}, Loss: {loss.item()}")

Streaming output truncated to the last 5000 lines.
Epoch: 5/10, Step: 629/938, Loss: 0.003361918032169342
Epoch: 5/10, Step: 630/938, Loss: 0.07565367966890335
Epoch: 5/10, Step: 631/938, Loss: 0.010830060578882694
Epoch: 5/10, Step: 632/938, Loss: 8.949342009145766e-05
Epoch: 5/10, Step: 633/938, Loss: 0.017498379573225975
Epoch: 5/10, Step: 634/938, Loss: 0.00018575586727820337
Epoch: 5/10, Step: 635/938, Loss: 0.013257138431072235
Epoch: 5/10, Step: 636/938, Loss: 0.0002461170661263168
Epoch: 5/10, Step: 637/938, Loss: 5.595730181084946e-05
Epoch: 5/10, Step: 638/938, Loss: 0.00014165250468067825
Epoch: 5/10, Step: 639/938, Loss: 0.0003704586415551603
Epoch: 5/10, Step: 640/938, Loss: 0.008190135471522808
Epoch: 5/10, Step: 641/938, Loss: 0.0004038750194013119
Epoch: 5/10, Step: 642/938, Loss: 0.002989268396049738
Epoch: 5/10, Step: 643/938, Loss: 0.0009864631574600935
Epoch: 5/10, Step: 644/938, Loss: 0.0019104795064777136
Epoch: 5/10, Step: 645/938, Loss: 0.008559586480259895
Epoc

In [ ]:
# Test the Model

with torch.no_grad():
  correct = 0
  total = 0

  for images, labels in test_loader:
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 99.07 %


In [ ]:
# Save state_dict
torch.save(model.state_dict(), "lenet_mnist.pth")